In [1]:
%%capture
# Capture because braindecode module prints a lot :S
from pyFiles.training import *
from pyFiles.models import *
from pyFiles.dataprep_down import *
from pyFiles.dataprep2_down import *
#
import torch
from torch import nn
from torch.nn import functional as F
#
from pyFiles.losses import InfoNceLoss, VICReg, BarlowTwins

In [2]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [3]:
#PREP DENUEVO
# prep("C:\\\\dataverse_files", mode = "per_win", save = True, save_dir = "data_down")

In [4]:
# spliter = DFSpliter(save = True, mode= "down_win")
# spliter("down_prep_windows.csv",".")

In [5]:
transforms = Augmentations(1, False, AUGMENTATIONS)

In [6]:
train_dataset = CustomEEGDataset("down_win_train_feats.csv", root_dir = "", transform= transforms)
val_dataset = CustomEEGDataset("down_win_val_feats.csv", root_dir = "",)

In [7]:
# trainloader = DataLoader(train_dataset, batch_size = 1, shuffle = True, num_workers = 0)
trainloader = DataLoader(train_dataset, batch_size = 10, shuffle = True, num_workers = 2)
valloader = DataLoader(val_dataset, batch_size = 10, shuffle = False, num_workers = 2)

In [8]:
model = SleepStager(n_chans = 19, pretext = False)
enc, fc = torch.load("params\multis\VR\\attempt2\\bestEncoderParamsmulti_VR.pt")
# enc, fc = torch.load("bestEncoderParamsmulti_VR.pt")
model.encoder.load_state_dict(enc)

criterion = nn.BCEWithLogitsLoss()
# criterion = nn.MSELoss()

#FREEZE
# for param in model.encoder.parameters():
#   param.requires_grad = False

# optimizer = torch.optim.Adam(model.encoder.parameters(), lr = 0.001)
optimizer = torch.optim.Adam(
    [
        {"params": model.encoder.parameters(), "lr": 1e-6},
        {"params": model.final.parameters(), "lr": 1e-3},
    ],
    lr=5e-4,
)
downtrain(model, 10, trainloader, valloader, criterion, optimizer, state = None, name = "multi_inf")

1824
Epoch nro 1/10
tensor([[-0.0741],
        [-0.0695],
        [-0.0616],
        [-0.0594],
        [-0.0681],
        [-0.0733],
        [-0.0784],
        [-0.0830],
        [-0.0740],
        [-0.0747]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.4815],
        [0.4826],
        [0.4846],
        [0.4852],
        [0.4830],
        [0.4817],
        [0.4804],
        [0.4793],
        [0.4815],
        [0.4813]], device='cuda:0', grad_fn=<SigmoidBackward0>)
Iter: 1/141, Loss:0.6786729097366333
tensor([[-0.3394],
        [-0.3279],
        [-0.3232],
        [-0.3251],
        [-0.3174],
        [-0.3131],
        [-0.2964],
        [-0.3037],
        [-0.3298],
        [-0.3238]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.4160],
        [0.4187],
        [0.4199],
        [0.4194],
        [0.4213],
        [0.4224],
        [0.4264],
        [0.4247],
        [0.4183],
        [0.4197]], device='cuda:0', grad_fn=<SigmoidBackward0>)
Iter: 2/141, Loss:0.771

KeyboardInterrupt: 

In [ ]:
model

SleepStager(
  (encoder): SleepStagerChambon2018(
    (spatial_conv): Conv2d(1, 19, kernel_size=(19, 1), stride=(1, 1))
    (feature_extractor): Sequential(
      (0): Conv2d(1, 16, kernel_size=(1, 50), stride=(1, 1), padding=(0, 25))
      (1): Identity()
      (2): ReLU()
      (3): MaxPool2d(kernel_size=(1, 13), stride=(1, 13), padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(16, 16, kernel_size=(1, 50), stride=(1, 1), padding=(0, 25))
      (5): Identity()
      (6): ReLU()
      (7): MaxPool2d(kernel_size=(1, 13), stride=(1, 13), padding=0, dilation=1, ceil_mode=False)
    )
  )
  (final): Sequential(
    (0): Linear(in_features=1824, out_features=1824, bias=True)
    (1): Dropout(p=0.4, inplace=False)
    (2): Linear(in_features=1824, out_features=100, bias=True)
    (3): Linear(in_features=100, out_features=1, bias=True)
  )
)